In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.combine import SMOTEENN
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [2]:
# Load dataset
df = pd.read_csv("nutrition_dataset_cleaned.csv")

# Fix duplicate goal columns
if 'Goal_Weight Maintenance' in df.columns and 'Goal_Maintenance' in df.columns:
    df['Goal_Maintenance'] = df[['Goal_Maintenance', 'Goal_Weight Maintenance']].max(axis=1)
    df = df.drop(columns=['Goal_Weight Maintenance'])

# Create new feature: BMI
df['BMI'] = df['Weight'] / (df['Height'] / 100) ** 2

# Merge low-sample categories
df['Breakfast Category'] = df['Breakfast Category'].replace(['yogurt-based', 'egg-based', 'pancake-based'], 'other')
le = LabelEncoder()
df['Breakfast Category Encoded'] = le.fit_transform(df['Breakfast Category'])

# Define features and target
X = df[['Age', 'Height', 'BMI', 'Diet_Omnivore', 'Diet_Vegan', 'Diet_Vegetarian', 'Goal_Muscle Gain', 'Goal_Weight Loss']]
y = df['Breakfast Category Encoded']

In [3]:
# Scale numerical features
scaler = StandardScaler()
X[['Age', 'Height', 'BMI']] = scaler.fit_transform(X[['Age', 'Height', 'BMI']])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTEENN
smoteenn = SMOTEENN(random_state=42, sampling_strategy='auto')
X_train, y_train = smoteenn.fit_resample(X_train, y_train)

# Initialize XGBoost
xgb_model = XGBClassifier(random_state=42)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9660\2863088.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['Age', 'Height', 'BMI']] = scaler.fit_transform(X[['Age', 'Height', 'BMI']])


In [7]:
# Hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 6],
    'subsample': [0.8, 1.0]
}
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='f1_weighted', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best model
best_xgb = grid_search.best_estimator_
print("\nBest XGBoost Parameters:", grid_search.best_params_)

# Predict and evaluate
y_pred = best_xgb.predict(X_test)
print("\nXGBoost Results:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0))

# Feature importance
feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': best_xgb.feature_importances_}).sort_values(by='Importance', ascending=False)
print("\nFeature Importance:")
print(feature_importance)


Best XGBoost Parameters: {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 100, 'subsample': 0.8}

XGBoost Results:
Accuracy: 0.6
Classification Report:
               precision    recall  f1-score   support

oatmeal-based       0.32      0.33      0.33        24
        other       0.69      0.54      0.60        41
   tofu-based       0.70      0.86      0.77        35

     accuracy                           0.60       100
    macro avg       0.57      0.58      0.57       100
 weighted avg       0.60      0.60      0.59       100


Feature Importance:
            Feature  Importance
3     Diet_Omnivore    0.484960
7  Goal_Weight Loss    0.169956
5   Diet_Vegetarian    0.121370
4        Diet_Vegan    0.088022
2               BMI    0.057072
1            Height    0.046321
6  Goal_Muscle Gain    0.023022
0               Age    0.009277


In [ ]:
# Save model
joblib.dump(best_xgb, "xgboost_optimized_breakfast_model.pkl")
print("\nXGBoost model saved as 'xgboost_optimized_breakfast_model.pkl'")